In [7]:
import torch
import torch.nn as nn
import numpy as np

In [50]:
class Network(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.batch_num = 0
        self.layers_dims = [397, 1024, 512, 256, 128, 96] #  5-layer model
        self.learning_rate = 0.001
        checkpoint = False
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(self.device)
        layers = []
        for n in range(len(self.layers_dims) - 2):
            layers.append(nn.Linear(self.layers_dims[n], self.layers_dims[n+1]))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(self.layers_dims[n+1], self.layers_dims[n+2]))
        layers.append(nn.Softmax(dim=1))
        self.model = nn.Sequential(*layers)
        self.apply(self._init_weights)
        self.activations = {}  # Dictionary to store activations
        self.hook_handles = []  # List to store hook handles
        self.model = self.model.to(self.device)


    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)

    def add_hooks(self):
        # Register hooks and store handles
        for name, layer in self.named_modules():
            handle = layer.register_forward_hook(self._save_activation(name))
            self.hook_handles.append(handle)

    def remove_hooks(self):
        # Remove all hooks using stored handles
        for handle in self.hook_handles:
            handle.remove()
        self.hook_handles.clear()  # Clear the list after removal

    def forward(self, x, nograd = True):
        x = self.convert(x)
        x = x.to(self.device)
        if nograd:
            with torch.no_grad():
                x = self.model(x)
                x = self.deconvert(x)
                return x
        else:
            return self.model(x)

    def _save_activation(self, name):
        # Hook function to save activations
        def hook(model, input, output):
            self.activations[name] = output.detach().cpu()
        return hook

    def convert(self, x):
        x = torch.from_numpy(np.array(x, dtype=np.float32)).transpose(0,1)
        return x

    def deconvert(self, x):
        if x.requires_grad:
            x = x.transpose(0,1).detach().numpy().astype(np.float64)
        else:
            x = x.transpose(0,1).numpy().astype(np.float64)
        return x
    

In [51]:
network = Network()

cpu


In [52]:
random = np.random.rand(397,50)
X = np.where(random >= 0.5, 1, 0)

In [53]:
Y = network(X)

In [26]:
print("Captured activations:", network.activations)

Captured activations: {}


In [28]:
print(Y.sum(1))

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


In [22]:
j[0,:] = m[0,:]
j[3,:] = m[3,:]

In [23]:
j

array([[10,  0],
       [ 0,  0],
       [ 0,  0],
       [ 6,  7]])

In [7]:
np.zeros((96))

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [45]:
z = np.zeros(20)
y = np.zeros(20)
z[0] = 1

In [46]:
if not np.array_equal(y, z):
    print('equal')

equal


In [37]:
z

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

In [38]:
y

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

In [48]:
layers_dims = [397, 1024, 512, 256, 128, 96]

In [52]:
for l in enumerate(layers_dims):
    print(l)

In [123]:
L = 5

In [124]:
for l in range (1, L):
    print(l)

1
2
3
4


In [129]:
red_parallel = np.zeros((10,10))
red_parallel_2 =np.ones((10,10))

In [127]:
red_parallel

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [128]:
red_parallel_batch = []

In [130]:
red_parallel_batch.append(red_parallel)
red_parallel_batch.append(red_parallel_2)
red_parallel_batch

[array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]),
 array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]

In [147]:
Y = np.hstack(red_parallel_batch)

In [148]:
print(Y)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]


In [149]:
yTorch = torch.from_numpy(Y)

In [151]:
yTorch = yTorch * 2

In [157]:
yTorch

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 2., 2., 2., 2., 2., 2., 2.,
         2., 2.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 2., 2., 2., 2., 2., 2., 2.,
         2., 2.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 2., 2., 2., 2., 2., 2., 2.,
         2., 2.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 2., 2., 2., 2., 2., 2., 2.,
         2., 2.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 2., 2., 2., 2., 2., 2., 2.,
         2., 2.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 2., 2., 2., 2., 2., 2., 2.,
         2., 2.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 2., 2., 2., 2., 2., 2., 2.,
         2., 2.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 2., 2., 2., 2., 2., 2., 2.,
         2., 2.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 2., 2., 2., 2., 2., 2., 2.,
         2., 2.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 2., 2., 2., 2., 2., 2., 2.,
         2., 2.]], dtype=tor

In [154]:
Y = Y * 2

In [156]:
Y

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2.]])

In [10]:
r = torch.rand(5,10)

In [11]:
r

tensor([[0.7302, 0.6091, 0.1549, 0.9017, 0.1234, 0.8782, 0.0391, 0.8252, 0.0432,
         0.1410],
        [0.5205, 0.8627, 0.4325, 0.9513, 0.7821, 0.6335, 0.5898, 0.4026, 0.9320,
         0.6654],
        [0.4595, 0.5178, 0.8663, 0.2911, 0.7448, 0.1995, 0.5466, 0.0338, 0.5519,
         0.9863],
        [0.5157, 0.6849, 0.8050, 0.6394, 0.1380, 0.7102, 0.9915, 0.2046, 0.5470,
         0.7592],
        [0.8816, 0.5947, 0.1256, 0.6725, 0.2499, 0.7425, 0.6869, 0.1677, 0.5164,
         0.1793]])

In [18]:
s = nn.Softmax(dim=1)

In [21]:
s(r).sum(1)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000])